In [ ]:
from diffusion_planner.data_process.data_processor import DataProcessor
from nuplan.common.actor_state.state_representation import Point2D
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_utils import ScenarioMapping
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_builder import NuPlanScenarioBuilder
from nuplan.planning.scenario_builder.scenario_filter import ScenarioFilter
from nuplan.planning.utils.multithreading.worker_parallel import SingleMachineParallelExecutor

from diffusion_planner.data_process.utils import (
get_scenario_map,
get_filter_parameters,
sampled_tracked_objects_to_tensor,
)

from diffusion_planner.data_process.roadblock_utils import (
  route_roadblock_correction
)


from diffusion_planner.data_process.agent_process import(sampled_tracked_objects_to_array_list,
                                                         sampled_static_objects_to_array_list,
                                                         agent_past_process,)
from diffusion_planner.data_process.map_process import(map_process,
                                                       get_neighbor_vector_set_map)
import numpy as np
import torch

In [3]:
db_path = "/cailiu2/Diffusion-Planner/data/2021.10.21.14.43.30_veh-28_01244_01519.db" # single db file
db_path = "/share/data_cold/open_data/nuplan/data/cache/mini"  # mini db files

map_path = "/share/data_cold/open_data/nuplan/maps"
save_path = "/cailiu2/Diffusion-Planner/data/processed"
map_version = "nuplan-maps-v1.0"

save_processed_path = "/cailiu2/Diffusion-Planner/data/processed"
total_scenarios = 10
scenario_mapping = ScenarioMapping(scenario_map=get_scenario_map(), subsample_ratio_override=0.5)
builder = NuPlanScenarioBuilder(db_path, map_path, None, None, map_version, scenario_mapping = scenario_mapping)
worker = SingleMachineParallelExecutor(use_process_pool=True)
scenario_filter = ScenarioFilter(*get_filter_parameters(num_scenarios_per_type=30000,
                                                          limit_total_scenarios=total_scenarios))
scenarios = builder.get_scenarios(scenario_filter, worker)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
del worker, builder, scenario_filter
processor = DataProcessor(scenarios,device)

Worker threads: 128


In [4]:
from diffusion_planner.data_process.utils import convert_absolute_quantities_to_relative,_global_state_se2_array_to_local,_global_velocity_to_local
from nuplan.planning.training.preprocessing.utils.agents_preprocessing import EgoInternalIndex, AgentInternalIndex


In [29]:
from nuplan.common.actor_state.tracked_objects_types import TrackedObjectType
scenario = processor._scenarios[0]
map_name = scenario._map_name
token = scenario.token
processor.scenario = scenario
processor.map_api = scenario.map_api

"""
ego
"""
ego_state = processor.scenario.initial_ego_state
ego_coords = Point2D(ego_state.rear_axle.x, ego_state.rear_axle.y)
anchor_ego_state = np.array([ego_state.rear_axle.x, ego_state.rear_axle.y, ego_state.rear_axle.heading], \
                            dtype=np.float64)

"""
neighbor agents
"""
neighbors = processor.scenario.initial_tracked_objects
neighbor_past = list(processor.scenario.get_past_tracked_objects(iteration= 0, time_horizon=1.0))
tracked_objects, tracked_objects_types, tracked_obj_tokens = sampled_tracked_objects_to_tensor(neighbors.tracked_objects)
#  tracked_objects_past, _ = sampled_tracked_objects_to_array_list(neighbor_past)
static_objects, static_objects_types =sampled_static_objects_to_array_list(neighbor_past[-1])
tracked_objects_list = []
for tracked_object in tracked_objects:
  tracked_objects_list.append(tracked_object)
_, neighbor_past, selected_indices, static_objects = agent_past_process(None, tracked_objects, tracked_objects_types, processor.num_agents,\
                                                        static_objects, static_objects_types, \
                                                        processor.num_static, processor.max_ped_bike, anchor_ego_state)
# print(selected_indices[12])

objects_tokens = [tracked_obj_tokens[key] for key in selected_indices]
# print(len(selected_indices))
# print(len(objects_tokens))
# print(objects_tokens)
neighbor_future = list(processor.scenario.get_future_tracked_objects(iteration= 0, time_horizon=8.0, num_samples =int(8*10)))
# print(len(neighbor_future))
slected_objs_future_traj = np.full((len(neighbor_future), len(objects_tokens), 11), np.nan, dtype=np.float64)
for frame_id in range(len(neighbor_future)):
  tracked_objects = neighbor_future[frame_id]
  all_future_neighbor_info = []
  current_frame_tracked_objects = {tracked_object.track_token: tracked_object for tracked_object in tracked_objects.tracked_objects}
  for i in range(len(objects_tokens)):
    if objects_tokens[i] in current_frame_tracked_objects.keys():
      agent_state = current_frame_tracked_objects[objects_tokens[i]]

      agent_global_poses = np.array([[agent_state.center.x, agent_state.center.y, agent_state.center.heading]])
      agent_global_velocities = np.array([[agent_state.velocity.x, agent_state.velocity.y]])
      transformed_poses = _global_state_se2_array_to_local(agent_global_poses, anchor_ego_state)
      transformed_velocities = _global_velocity_to_local(agent_global_velocities, anchor_ego_state[-1])
      local_agent_state = np.zeros((1, 11))
      local_agent_state[:, 0] = transformed_poses[:, 0]
      local_agent_state[:, 1] = transformed_poses[:, 1]
      local_agent_state[:, 2] = np.cos(transformed_poses[:, 2])
      local_agent_state[:, 3] = np.sin(transformed_poses[:, 2])
      local_agent_state[:, 4] = transformed_velocities[:, 0]
      local_agent_state[:, 5] = transformed_velocities[:, 1]
      local_agent_state[:, 6] = agent_state.box.width
      local_agent_state[:, 7] = agent_state.box.length
      if agent_state.tracked_object_type == TrackedObjectType.VEHICLE:
        local_agent_state[:, 8:] = [1, 0, 0]  # Mark as VEHICLE
      elif agent_state.tracked_object_type == TrackedObjectType.PEDESTRIAN:
        local_agent_state[:, 8:] = [0, 1, 0]  # Mark as PEDESTRIAN
      else:  # TrackedObjectType.BICYCLE
        local_agent_state[:, 8:] = [0, 0, 1]  # Mark as BICYCLE
      slected_objs_future_traj[frame_id, i, :] = local_agent_state
# selected_neighbor_future = np.full(len(objects_tokens))
slected_objs_future_traj = slected_objs_future_traj.transpose(1, 0, 2)
print(neighbor_past.shape)
print(slected_objs_future_traj.shape)
# neighbor_future.tracked_objects
# for token in objects_tokens:
#   selected_neighbor_future.append([selected_tracked_object for selected_tracked_object in neighbor_future \
#                                   if selected_tracked_object.tracked_objects.track_token == token])

(30, 1, 11)
(30, 80, 11)


In [30]:
future_ego_states = processor.scenario.get_ego_future_trajectory(iteration=0, num_samples=int(8*10), time_horizon= 8.0)
future_ego_states_numpy = np.array([(ego_state.rear_axle.x, ego_state.rear_axle.y, ego_state.rear_axle.heading, \
                  ego_state.dynamic_car_state.rear_axle_velocity_2d.x, ego_state.dynamic_car_state.rear_axle_velocity_2d.y,\
                  ego_state.dynamic_car_state.rear_axle_acceleration_2d.x, ego_state.dynamic_car_state.rear_axle_acceleration_2d.y ) for ego_state in future_ego_states], dtype = np.float64)
local_future_ego_states = convert_absolute_quantities_to_relative(future_ego_states_numpy, anchor_ego_state,'ego')

In [3]:
processor.work(save_dir=save_processed_path, debug = False )

  0%|          | 0/10 [00:09<?, ?it/s]


ValueError: too many values to unpack (expected 2)

In [5]:
"""
ego
"""
ego_state = processor.scenario.initial_ego_state
ego_coords = Point2D(ego_state.rear_axle.x, ego_state.rear_axle.y)
anchor_ego_state = np.array([ego_state.rear_axle.x, ego_state.rear_axle.y, ego_state.rear_axle.heading], dtype=np.float64)
print(ego_coords)
print(anchor_ego_state)


"""
neighbor
"""
# neighbor_agent_past, neighbor_agents_types = processor.get_neighbor_agents()
neighbors = processor.scenario.initial_tracked_objects
neighbors.tracked_objects
# print(len(neighbors.tracked_objects))
# print(type(neighbors.tracked_objects))

neighbor_past = list(processor.scenario.get_past_tracked_objects(iteration= 0, time_horizon=1.0))
print(type(neighbor_past))
print(type(neighbor_past[0]))
# for neighbor in neighbors:
tracked_objects, tracked_objects_types = sampled_tracked_objects_to_tensor(neighbors.tracked_objects)
tracked_objects_past, _ = sampled_tracked_objects_to_array_list(neighbor_past)
static_objects, static_objects_types =sampled_static_objects_to_array_list(neighbor_past[-1])
# print(type(neighbor_past))
# print(len(neighbor_past))
tracked_objects_list = []

for tracked_object in tracked_objects:
  tracked_objects_list.append(tracked_object)
print(tracked_objects_past[0].shape)
print(len(tracked_objects_past))
print(len(static_objects))

num_agents = 30
num_statics = 30
max_ped_bike = 10
_, neighbor_past, _, static_objects = agent_past_process(None, tracked_objects, tracked_objects_types, num_agents,static_objects, static_objects_types, \
                                                          num_statics, max_ped_bike, anchor_ego_state)
# print(len(tracked_objects_list))


Point2D(x=330957.49299200956, y=4691143.579406448)
[3.30957493e+05 4.69114358e+06 5.43869085e-01]
<class 'list'>
<class 'nuplan.planning.simulation.observation.observation_type.DetectionsTracks'>
(57, 8)
10
4


In [11]:
'''
Map
'''
map_api = processor.scenario.map_api
route_roadblocks_ids = processor.scenario.get_route_roadblock_ids()

route_roadblocks_ids =route_roadblock_correction(
                    ego_state, map_api, route_roadblocks_ids
                    )
traffic_light_data = processor.scenario.get_traffic_light_status_at_iteration(iteration= 0)
coords, traffic_light_data, speed_limit, lane_route =get_neighbor_vector_set_map(
            map_api, processor._map_features, ego_coords, processor._radius, traffic_light_data)
vector_map = map_process(route_roadblocks_ids, anchor_ego_state, coords, traffic_light_data, speed_limit, lane_route, processor._map_features,
                            processor._max_elements, processor._max_points)


In [25]:
data = {"neighbor_agents_past": neighbor_past[:, -21:],
        "ego_current_state": np.array([0., 0., 1. ,0.], dtype=np.float32), # ego centric x, y, cos, sin
        "static_objects": static_objects}
data.update(vector_map)
import torch
from diffusion_planner.data_process.utils import convert_to_model_inputs
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data = convert_to_model_inputs(data, device)


In [26]:
import os

# 处理时间格式

save_path = "/cailiu2/Diffusion-Planner/data/processed"
# 生成文件名
file_name = f"{processor.scenario.log_name}_{processor.scenario.token}_{processor.scenario.scenario_type}.pt"

# 确保文件名合法（移除特殊字符）
file_name = file_name.replace(":", "_").replace("/", "_").replace(" ", "_")

# 指定存放路径
os.makedirs(save_path, exist_ok=True)  # 确保目录存在

# 完整文件路径
file_path = os.path.join(save_path, file_name)
torch.save(data, file_path)


In [1]:
import torch
print(torch.cuda.device_count())  # 如果为 True，表示有 CUDA 可用；如果为 False，则没有 CUDA 可用
print(torch.__version__)

2
1.12.1+cu113


/root/miniconda3/envs/dlp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
